# Dyson Protocol Nameservice Guide

The Nameservice module represents a cornerstone of the Dyson Protocol ecosystem, providing a sophisticated decentralized identity and asset management system. This comprehensive guide walks you through the elegant architecture of name registration, NFT integration, and token creation—demonstrating how these components harmoniously interact to enable a new paradigm of digital ownership. By following these carefully crafted examples, you'll gain hands-on experience with the full spectrum of Nameservice capabilities, from secure two-phase name registration to advanced NFT collection management and custom asset creation.

## Fetch Your Address

First, we'll retrieve the addresses associated with the 'alice' and 'bob' accounts. These addresses will serve as our identities throughout this guide.

In [1]:
[alice_address] = ! dysond keys show alice -a
[bob_address] = ! dysond keys show bob -a
print(f"Alice address: {alice_address}")
print(f"Bob address: {bob_address}")

Alice address: dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz
Bob address: dys1fhhxp9xveswc4yhxekr32eqe80rkwpurya0jh0


## Check Nameservice Parameters

Let's examine the current parameters of the nameservice module to understand the rules for bidding, valuations, and accepted denominations.

In [2]:
! dysond query nameservice params -o json | jq -M

{
  "params": {
    "bid_timeout": "2s",
    "allowed_denoms": [
      "dys"
    ],
    "reject_bid_valuation_fee_percent": "0.03",
    "minimum_bid_percent_increase": "0.01"
  }
}


## Name Registration Process

Registering a name in Dyson Protocol uses a two-step commit-reveal process to prevent front-running. Let's register a name following this process.

### Generate Salt and Name

First, let's prepare a name and generate a random salt value for the commitment.

In [3]:
import random
import string
import json

def random_string(length=10):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))


name = f"alice-{random_string(5)}.dys"
salt = random_string(20)


print(f"Name: {name}")
print(f"Salt: {salt}")

Name: alice-px4fp.dys
Salt: xamah75hd9wsbsutf5j5


### Compute Hash for Commitment

Now, we'll compute a hash using the name, salt, and committer address. This hash will be used in the commitment phase.

In [4]:
[name_commit_hex_hash] = ! dysond query nameservice compute-hash \
    --name "$name" \
    --salt "$salt" \
    --committer "$alice_address"  -o json| jq '.hex_hash' -r
print(f"Hex Hash: {name_commit_hex_hash}")

Hex Hash: 7ff904347206b645808bc57c164436a63099fcf5349ea8ba9493e7848d0b1e09


### Commit Phase

In this first phase, we commit to registering the name by submitting the hash and setting an initial valuation.

In [5]:
valuation = '100dys'
[txhash] = ! dysond tx nameservice commit \
    --commitment "$name_commit_hex_hash" \
    --valuation "$valuation" \
    --from alice -y -o json | jq -r .txhash ; sleep 0.01
# Some delay to ensure the transaction is available

print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
print(tx_result)
tx_result = json.loads("".join(tx_result))

assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"
print(f"Tx error code: {tx_result['code']}")

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: 403B752EFA07A7265A923B6A001701A2D6A4CBE5F19B3D486AED8767EAE7A7D8


['{"height":"371402","txhash":"403B752EFA07A7265A923B6A001701A2D6A4CBE5F19B3D486AED8767EAE7A7D8","codespace":"","code":0,"data":"12310A2F2F6479736F6E70726F746F636F6C2E6E616D65736572766963652E76312E4D7367436F6D6D6974526573706F6E7365","raw_log":"","logs":[],"info":"","gas_wanted":"200000","gas_used":"39847","tx":null,"timestamp":"","events":[{"type":"tx","attributes":[{"key":"acc_seq","value":"dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz/155","index":true}]},{"type":"tx","attributes":[{"key":"signature","value":"IbhZCbv+0ruJcnzVrXCmPoOI1dg8/J1aFlWeTr461Xkmr2WYZ+u1T9MzZZDn+NTUgtdV91m3RQyq0Hn5HRBkhQ==","index":true}]},{"type":"message","attributes":[{"key":"action","value":"/dysonprotocol.nameservice.v1.MsgCommit","index":true},{"key":"sender","value":"dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz","index":true},{"key":"module","value":"nameservice","index":true},{"key":"msg_index","value":"0","index":true}]},{"type":"dysonprotocol.nameservice.v1.EventCommitmentCreated","attributes":[{"key":"he

### Reveal Phase

In the second phase, we reveal the actual name and salt to complete the registration process.

In [6]:
[txhash] = ! dysond tx nameservice reveal \
    --name "$name" \
    --salt "$salt" \
    --from alice \
    -y -o json | jq -r .txhash ; sleep 0.01

print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: FA20FC7C16C25A0738F37ABF59547BB0BB6C31966904C8AEEEA3F25D142CA37D


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventNameRegistered",
  "attributes": [
    {
      "key": "fee",
      "value": "[{\"denom\":\"dys\",\"amount\":\"1\"}]",
      "index": true
    },
    {
      "key": "name",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Verify Name Registration

Let's verify that the name was properly registered by querying the NFT details. Each registered name becomes an NFT in the 'nameservice.dys' class.

In [7]:
! dysond query nft nft "nameservice.dys" "$name" 

{
  "nft": {
    "class_id": "nameservice.dys",
    "id": "alice-px4fp.dys",
    "uri": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz",
    "data": {
      "type": "/dysonprotocol.nameservice.v1.NFTData",
      "value": {
        "listed": true,
        "valuation": {
          "denom": "dys",
          "amount": "100"
        },
        "valuation_expiry": "2026-06-06T10:46:13.698588Z",
        "current_bid": {
          "amount": "0"
        }
      }
    }
  }
}


## Name Destination Management

Setting a destination for a name allows it to resolve to a specific address, enabling service discovery within the Dyson Protocol ecosystem.

In [8]:
[txhash] = ! dysond tx nameservice set-destination \
    --name "$name" \
    --destination "$alice_address" \
    --from alice \
    -y -o json | jq -r .txhash ; sleep 0.01
# Some delay to ensure the transaction is available

print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: AC87672621CB20675AB777FAA30BC425A8BF0B2C65AB8FF3636F4DF2D769E04B


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventNameDestinationSet",
  "attributes": [
    {
      "key": "destination",
      "value": "\"dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz\"",
      "index": true
    },
    {
      "key": "name",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Verify Destination Setting

Let's confirm that the destination was correctly set by checking the NFT's URI field, which stores the destination address.

In [9]:
! dysond query nft nft nameservice.dys "$name"

{
  "nft": {
    "class_id": "nameservice.dys",
    "id": "alice-px4fp.dys",
    "uri": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz",
    "data": {
      "type": "/dysonprotocol.nameservice.v1.NFTData",
      "value": {
        "listed": true,
        "valuation": {
          "denom": "dys",
          "amount": "100"
        },
        "valuation_expiry": "2026-06-06T10:46:13.698588Z",
        "current_bid": {
          "amount": "0"
        }
      }
    }
  }
}


# Update your script to serve the DWapp
Use the following command to update the script to serve the DWapp.

In [10]:

[txhash] = ! dysond tx script update --code-path "../examples/simple_wsgi_example.py" \
    --from alice \
    -y -o json | jq -r .txhash

print(f"Transaction hash: {txhash}")


Transaction hash: F301475558052B2D619832A00965CB121D70D6107874C650837E4D8401B71511


In [11]:
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Tx error code: 0
{
  "type": "dysonprotocol.script.v1.EventUpdateScript",
  "attributes": [
    {
      "key": "version",
      "value": "\"21\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


# Accessing your DWapp

Note that the DWapp is a simple WSGI app that can be accessed at the address of the name you registered.

For example, if you registered the name "alice.dys", you can access the DWapp at "http://alice.dys.localhost:8000"

All requests to the DWapp are queries to the blockchain state, not transactions. So you can't change the state of the blockchain through the DWapp even with a POST request.

In [12]:
[output] = ! dysond config get app api.address
port = output.split(":")[-1].strip("\"")

dwapp_url = f"http://{name}.localhost:{port}"

print(f"=== Making a GET request to your DWapp at '{dwapp_url}' ===")
out = ! curl -s "$dwapp_url/hi?name=bob"
out = "\n".join(out).strip()
print(out)

assert "hi bob" in out, "Expected 'hi bob' in output, got: " + out
assert "Request Method: GET" in out, "Expected 'Request Method: GET' in output, got: " + out

print()
print(f"=== Making a POST request to your DWapp at '{dwapp_url}' ===")
out = ! curl -s -X POST "$dwapp_url/hi" -d "name=bob"
out = "\n".join(out).strip()
print(out)

assert "hi bob" in out, "Expected 'hi bob' in output, got: " + out
assert "Request Method: POST" in out, "Expected 'Request Method: POST' in output, got: " + out

=== Making a GET request to your DWapp at 'http://alice-px4fp.dys.localhost:2417' ===


hi bob

Request Method: GET
Query String: name=bob
Path Info: /hi

=== Making a POST request to your DWapp at 'http://alice-px4fp.dys.localhost:2417' ===


hi bob

Request Method: POST
Query String: 
Path Info: /hi
Content Type: application/x-www-form-urlencoded
Content Length: 8


## Name Valuation Management

Names in the system have a value, which act as a starting point for bids when trading names. Let's update the valuation for our name.

In [13]:

new_valuation = "200dys"

[txhash] =! dysond tx nameservice set-valuation \
    --class-id="nameservice.dys" \
    --nft-id="$name" \
    --valuation="$new_valuation" \
    --from=alice \
    -y \
    -o json | jq -r .txhash ; sleep 0.01

print(f"Transaction hash: {txhash}")


Transaction hash: A1B7E8E9E2CF5DBDD250B926D6F196A9BF18C6B406FD184A5FE2F262C45420F1


In [14]:

tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventNameValuationUpdated",
  "attributes": [
    {
      "key": "name",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "new_valuation",
      "value": "{\"denom\":\"dys\",\"amount\":\"200\"}",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Verify Updated Valuation

Let's confirm the updated valuation by querying the NFT data.

In [15]:
! dysond query nft nft "nameservice.dys" "$name" 

{
  "nft": {
    "class_id": "nameservice.dys",
    "id": "alice-px4fp.dys",
    "uri": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz",
    "data": {
      "type": "/dysonprotocol.nameservice.v1.NFTData",
      "value": {
        "listed": true,
        "valuation": {
          "denom": "dys",
          "amount": "200"
        },
        "valuation_expiry": "2026-06-06T10:46:13.698588Z",
        "current_bid": {
          "amount": "0"
        }
      }
    }
  }
}


## NFT Collection Creation

One of the powerful features of the nameservice module is the ability to create NFT collections under your registered name.

### Create Main NFT Class

Let's create a main NFT collection using our registered name as the class ID.

In [16]:
[txhash] = ! dysond tx nameservice save-class \
        --class-id=$name \
        --from=alice \
        --name="Main Collection" \
        --symbol=MAINCOL \
        --description="My Main Collection" \
        --uri=https://example.com/main \
        -y | dysond query wait-tx -o json | jq -r .txhash ; sleep 0.01

print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: D5947E03A4B758F940D4F5EE5CD5EA175A28192D872B63AA4FEAE2536520354C


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventClassSaved",
  "attributes": [
    {
      "key": "class_id",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Create a Sub-Collection

We can also create sub-collections under our main collection by using a hierarchical class ID.

In [17]:
subcollection_id = f"{name}/subcollection"
tx = ! dysond tx nameservice save-class --class-id=$subcollection_id --name="Sub Collection" --symbol=SUBCOL --description="My Sub-Collection" --uri=https://example.com/sub --from=alice -y -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: D67974A1CC3A4E94F7BE849A3773DEDFF65E85E7B1E6F00255EEB8A3BD82CE39


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventClassSaved",
  "attributes": [
    {
      "key": "class_id",
      "value": "\"alice-px4fp.dys/subcollection\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### View All NFT Classes

Let's view all the NFT classes in the system to confirm our collections were created successfully.

In [18]:
! dysond query nft classes -o json | jq -M

{
  "classes": [
    {
      "id": "alice-hlqlh.dys",
      "name": "Main Collection",
      "symbol": "MAINCOL",
      "description": "My Main Collection",
      "uri": "https://example.com/main",
      "data": {
        "type": "/dysonprotocol.nameservice.v1.NFTClassData",
        "value": {
          "extra_data": "{\"website\": \"https://example.com/details\"}"
        }
      }
    },
    {
      "id": "alice-hlqlh.dys/subcollection",
      "name": "Sub Collection",
      "symbol": "SUBCOL",
      "description": "My Sub-Collection",
      "uri": "https://example.com/sub"
    },
    {
      "id": "alice-o3pyh.dys",
      "name": "Main Collection",
      "symbol": "MAINCOL",
      "description": "My Main Collection",
      "uri": "https://example.com/main",
      "data": {
        "type": "/dysonprotocol.nameservice.v1.NFTClassData",
        "value": {
          "extra_data": "{\"website\": \"https://example.com/details\"}"
        }
      }
    },
    {
      "id": "alice-o3pyh.dys

## NFT Minting

Now that we have created NFT collections, let's mint some NFTs within these collections.

In [19]:
# Mint an NFT in the main collection
nft_id = "nft1"
tx = ! dysond tx nameservice mint-nft \
    --class-id=$name \
    --nft-id=$nft_id \
    --uri=https://example.com/nft1 \
    --from=alice \
    -y \
    -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: FE12C64182E47DDB773EB7CCD2BAC35DA14F2DF340D9ADB7C1B3928B0E2664A1


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventNFTMinted",
  "attributes": [
    {
      "key": "class_id",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "nft_id",
      "value": "\"nft1\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


In [20]:
# Mint an NFT in the sub-collection
subnft_id = f"subnft1-{random_string(5)}"
tx = ! dysond tx nameservice mint-nft --class-id=$subcollection_id --nft-id=$subnft_id --uri=https://example.com/subnft1 --from=alice -y -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: 3402A69850DA5F9225D50A22964A8E05D3002A5F1295C7E374E17FBE45F1A1C9


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventNFTMinted",
  "attributes": [
    {
      "key": "class_id",
      "value": "\"alice-px4fp.dys/subcollection\"",
      "index": true
    },
    {
      "key": "nft_id",
      "value": "\"subnft1-8jpok\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### View NFTs in Collection

Let's verify the NFTs in our main collection.

In [21]:
! dysond query nft nfts $name -o json | jq -M

{
  "nfts": [
    {
      "class_id": "alice-px4fp.dys",
      "id": "nft1",
      "uri": "https://example.com/nft1"
    }
  ],
  "pagination": {
    "total": "1"
  }
}


# Verify the NFTs in the sub-collection


In [22]:
! dysond query nft nfts $subcollection_id -o json | jq -M

{
  "nfts": [
    {
      "class_id": "alice-px4fp.dys/subcollection",
      "id": "subnft1-8jpok",
      "uri": "https://example.com/subnft1"
    }
  ],
  "pagination": {
    "total": "1"
  }
}


## NFT Metadata Management

NFTs can have additional text metadata to describe their properties and attributes. Let's add metadata to our NFT.

In [23]:
import json
import shlex

# Set metadata for the NFT, note that the metadata should be escaped for the shell

metadata = shlex.quote(json.dumps({"some_key":"some_value", "another_key": "He doesn't eat his vegetables"}))
print(f"Metadata: {metadata}")
print(f"Alice address: {alice_address}")

tx = ! dysond tx nameservice set-nft-metadata \
    --from=$alice_address \
    --class-id="$name" \
    --nft-id="$nft_id" \
    --metadata=$metadata \
    -y \
    -o json | jq .txhash -r ; sleep 0.01

txhash = tx[0]
print(f"Transaction hash: {txhash}")

Metadata: '{"some_key": "some_value", "another_key": "He doesn'"'"'t eat his vegetables"}'
Alice address: dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz


Transaction hash: 14909EC4F1F9AC0F9F1B954F7076F20E678C0368D2C62819BDBB89DECE1ADBB5


In [24]:
tx_result = ! dysond query wait-tx "$txhash" -o json
tx_result = json.loads("".join(tx_result))
print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

# Verify the NFT with metadata
! dysond query nft nft $name $nft_id

Tx error code: 0


{
  "nft": {
    "class_id": "alice-px4fp.dys",
    "id": "nft1",
    "uri": "https://example.com/nft1",
    "data": {
      "type": "/dysonprotocol.nameservice.v1.NFTData",
      "value": {
        "valuation": {
          "amount": "0"
        },
        "valuation_expiry": "0001-01-01T00:00:00Z",
        "current_bid": {
          "amount": "0"
        },
        "metadata": "{\"some_key\": \"some_value\", \"another_key\": \"He doesn't eat his vegetables\"}"
      }
    }
  }
}


### Add Extra Data to NFT Class

We can also add additional data to the NFT class itself to provide more information about the collection.

In [25]:
# Set extra data for the NFT class

extra_data = shlex.quote(json.dumps({"website":"https://example.com/details"}))

tx = ! dysond tx nameservice set-nft-class-extra-data --class-id=$name --extra-data=$extra_data --from=alice -y -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))
print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

# Verify the NFT class with extra data
print("NFT class with new extra data:")
! dysond query nft class $name -o json

Transaction hash: 65237473E27089102971514F2D9719C0ECAE908F4E4EC88ECD5DE7698D89C65B


Tx error code: 0
NFT class with new extra data:


{
  "class": {
    "id": "alice-px4fp.dys",
    "name": "Main Collection",
    "symbol": "MAINCOL",
    "description": "My Main Collection",
    "uri": "https://example.com/main",
    "data": {
      "type": "/dysonprotocol.nameservice.v1.NFTClassData",
      "value": {
        "extra_data": "{\"website\": \"https://example.com/details\"}"
      }
    }
  }
}


## Custom Coin Operations

Dyson Protocol allows name owners to mint custom coins using their registered names as denominations.

### Mint Coins with Name Denomination

Let's mint some coins using our registered name as the denomination.

In [26]:
# Mint coins with the name as denomination
amount = f"1000{name}"
tx = ! dysond tx nameservice mint-coins --amount=$amount --from=alice -y -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: 3CE48704A2B44134FBEA7B251F82F60BD62D644CEBD26609EF5D2B63B5D272FB


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventCoinsMinted",
  "attributes": [
    {
      "key": "amount",
      "value": "[{\"denom\":\"alice-px4fp.dys\",\"amount\":\"1000\"}]",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Mint Coins with Subdenom

We can also mint coins with subdenominations for more specific token creation.

In [27]:
# Mint coins with a subdenom
subdenom = f"{name}/token1"
amount = f"500{subdenom}"
tx = ! dysond tx nameservice mint-coins --amount=$amount --from=alice -y -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: A9095B6DE2A97AD36C81C0A265D3C872B4707725A0B36D8E0D6E4C40CFDF7EB4


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventCoinsMinted",
  "attributes": [
    {
      "key": "amount",
      "value": "[{\"denom\":\"alice-px4fp.dys/token1\",\"amount\":\"500\"}]",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Check Balance

Let's check Alice's balance to confirm the minted coins have been added to her account.

In [28]:
! dysond query bank balances $alice_address -o json | jq -M

{
  "balances": [
    {
      "denom": "alice-hlqlh.dys",
      "amount": "800"
    },
    {
      "denom": "alice-hlqlh.dys/token1",
      "amount": "500"
    },
    {
      "denom": "alice-o3pyh.dys",
      "amount": "800"
    },
    {
      "denom": "alice-o3pyh.dys/token1",
      "amount": "500"
    },
    {
      "denom": "alice-px4fp.dys",
      "amount": "1000"
    },
    {
      "denom": "alice-px4fp.dys/token1",
      "amount": "500"
    },
    {
      "denom": "alice-s52fn.dys",
      "amount": "800"
    },
    {
      "denom": "alice-s52fn.dys/token1",
      "amount": "500"
    },
    {
      "denom": "alice-z82xb.dys",
      "amount": "800"
    },
    {
      "denom": "alice-z82xb.dys/token1",
      "amount": "500"
    },
    {
      "denom": "dys",
      "amount": "9998935777"
    }
  ],
  "pagination": {
    "total": "11"
  }
}


### Transfer Custom Coins

Now that we have minted custom coins, let's send some to Bob's account.

In [29]:
# Send custom coins to Bob
transfer_amount = f"200{name}"
tx = ! dysond tx bank send alice $bob_address $transfer_amount -y -o json | jq -r .txhash
txhash = tx[0]
print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: C8461349664AAB5DDC137956A043E0A47F431983303512EB56CEC7670370DF52


Tx error code: 0


### Verify Bob's Balance

Let's check Bob's balance to confirm the transfer was successful.

In [30]:
! dysond query bank balances $bob_address -o json | jq

{
  "balances": [
    {
      "denom": "alice-hlqlh.dys",
      "amount": "200"
    },
    {
      "denom": "alice-o3pyh.dys",
      "amount": "200"
    },
    {
      "denom": "alice-px4fp.dys",
      "amount": "200"
    },
    {
      "denom": "alice-s52fn.dys",
      "amount": "200"
    },
    {
      "denom": "alice-z82xb.dys",
      "amount": "200"
    },
    {
      "denom": "dys",
      "amount": "9999999183"
    }
  ],
  "pagination": {
    "total": "6"
  }
}


## Name Trading Process

The Nameservice module allows names to be traded through a secure bidding system. Let's demonstrate how Bob can bid on Alice's name.

In [31]:
print(f"Bob places a bid on Alice's name: {name}")

current_bid_result = ! dysond query nft nft nameservice.dys $name -o json 
current_bid_result = json.loads("".join(current_bid_result))
print(f"Current bid result: {current_bid_result}")
current_bid = current_bid_result['nft']['data']['value']['current_bid']

current_valuation = current_bid_result['nft']['data']['value']['valuation']

print(f"Current bid: {current_bid}") # Current bid: {'amount': '0'}
print(f"Current valuation: {current_valuation}") # Current valuation: {'amount': '100'}


min_bid_amount = max(int(current_bid['amount']) + 100, int(current_valuation['amount'])) 
bid_amount = f"{min_bid_amount}dys"
print(f"Bob's bid amount: {bid_amount}")
#  dysond tx nameservice place-bid --nft-class-id=<class-id> --nft-id=<nft-id> --bid-amount=<amount> [flags]
[txhash] = ! dysond tx nameservice place-bid \
    --nft-class-id="nameservice.dys" \
    --nft-id=$name \
    --bid-amount=$bid_amount \
    --from=bob \
    -y \
    -o json | jq -r .txhash

print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Bob places a bid on Alice's name: alice-px4fp.dys


Current bid result: {'nft': {'class_id': 'nameservice.dys', 'id': 'alice-px4fp.dys', 'uri': 'dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz', 'data': {'type': '/dysonprotocol.nameservice.v1.NFTData', 'value': {'listed': True, 'valuation': {'denom': 'dys', 'amount': '200'}, 'valuation_expiry': '2026-06-06T10:46:13.698588Z', 'current_bid': {'amount': '0'}}}}}
Current bid: {'amount': '0'}
Current valuation: {'denom': 'dys', 'amount': '200'}
Bob's bid amount: 200dys


Transaction hash: C5AF1AC3EBF6C419CAED8F48923D624515887EBA40EC1DA80E38754816C036F6


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventBidPlaced",
  "attributes": [
    {
      "key": "bid_amount",
      "value": "{\"denom\":\"dys\",\"amount\":\"200\"}",
      "index": true
    },
    {
      "key": "bidder",
      "value": "\"dys1fhhxp9xveswc4yhxekr32eqe80rkwpurya0jh0\"",
      "index": true
    },
    {
      "key": "class_id",
      "value": "\"nameservice.dys\"",
      "index": true
    },
    {
      "key": "nft_id",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Verify Current Bid

Let's check the current bid on the name.

In [32]:
! dysond query nft nft nameservice.dys $name -o json | jq -M '.nft.data.value.current_bid' 

{
  "denom": "dys",
  "amount": "200"
}


### Accept Bid

Alice can choose to accept Bob's bid, which will transfer the name to Bob and the bid amount to Alice.

In [33]:
# Alice accepts Bob's bid
#   dysond tx nameservice accept-bid --nft-class-id=<class-id> --nft-id=<nft-id> [flags]

tx = ! dysond tx nameservice accept-bid \
    --nft-class-id="nameservice.dys" \
    --nft-id=$name \
    --from=alice \
    -y \
    -o json | jq -r .txhash

txhash = tx[0]

print(f"Transaction hash: {txhash}")
tx_result = ! dysond query wait-tx $txhash -o json
tx_result = json.loads("".join(tx_result))

print(f"Tx error code: {tx_result['code']}")
assert tx_result['code'] == 0, f"Tx failed with code {tx_result['code']}, {tx_result['raw_log']}"

for event in tx_result['events']:
    if 'dysonprotocol' in event['type']:
        print(json.dumps(event, indent=2))


Transaction hash: 9165C06BE79781D632F78C83CFCFAB1083F2BCADD788FA123D50328F644D63FD


Tx error code: 0
{
  "type": "dysonprotocol.nameservice.v1.EventBidAccepted",
  "attributes": [
    {
      "key": "class_id",
      "value": "\"nameservice.dys\"",
      "index": true
    },
    {
      "key": "new_owner",
      "value": "\"dys1fhhxp9xveswc4yhxekr32eqe80rkwpurya0jh0\"",
      "index": true
    },
    {
      "key": "nft_id",
      "value": "\"alice-px4fp.dys\"",
      "index": true
    },
    {
      "key": "msg_index",
      "value": "0",
      "index": true
    }
  ]
}


### Verify Name Ownership

Let's verify that the name has been transferred to Bob.

In [34]:
out = ! dysond keys show bob -a
bob_address = "\n".join(out).strip()
print(f"Bob's address: {out}")

out = ! dysond query nft owner nameservice.dys $name -o json | jq 
out = "\n".join(out).strip()
nft_owner_data = json.loads(out)
nft_owner = nft_owner_data['owner']
print(nft_owner)

assert nft_owner == bob_address, "Expected 'alice' in output, got: " + nft_owner

Bob's address: ['dys1fhhxp9xveswc4yhxekr32eqe80rkwpurya0jh0']


dys1fhhxp9xveswc4yhxekr32eqe80rkwpurya0jh0


## Conclusion

This guide has demonstrated the key features of the Dyson Protocol Nameservice Module. We've covered:

1. Name registration through a commit-reveal process
2. Setting name destinations for resolution
3. Creating NFT collections and minting NFTs
4. Managing metadata for NFTs and collections
5. Minting custom coins with name-based denominations
6. Trading names through a bidding system

These capabilities enable a powerful decentralized namespace system that integrates with NFTs and custom tokens, forming a foundation for various applications on the Dyson Protocol blockchain.